# Summary Tables
This notebook will create summaries of the data with the following format:

![wanted_output](format2.png)

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns

# Read in the data
df = pd.read_csv('../results/all_stats_species_2023-03-14.csv')
df = df.loc[df["threshold"] == 0.0001]
# If the source is mixed, hilo, or tourlousse, then we need to average the samples together.
replicate_comm = ["mixed", "hilo", "tourlousse"]

rename_dict = {"hilo": "Amos HiLo", "mixed": "Amos Mixed", "tourlousse": "Tourlousse"}
metrics = ["FPRA", "Sens", "AD"]
wanted_pipelines = ["biobakery3", "biobakery4", "jams", "wgsa2", "woltka"]
wanted_cols_one_to_one = metrics + ["Community", "Pipeline"]
wanted_cols_replicates = metrics + ["Pipeline", "Source"]

replicates = df.loc[df["Source"].isin(replicate_comm) & df["Pipeline"].isin(wanted_pipelines)]
one_to_one = df.loc[~df["Source"].isin(replicate_comm) & df["Pipeline"].isin(wanted_pipelines)]

# Average together replicates within same source and pipeline
replicates = replicates.groupby(["Pipeline", "Source"]).agg({
    "FPRA": ["count", "mean", "std"],
    "Sens": ["count", "mean", "std"],
    "AD": ["count", "mean", "std"],
})

# display(replicates)

replicates = replicates.groupby(["Source", "Pipeline"]).mean().reset_index()
replicates = replicates.loc[:, wanted_cols_replicates]

for metric in metrics:
    test = replicates[["Pipeline", "Source", metric]]
    test = test.droplevel(0, axis=1)
    test.columns = ["Pipeline", "Source", "count", "mean", "std"]
    test.index.name = metric
    test = test.pivot(index="Source", columns="Pipeline", values=["count", "mean", "std"])
    # Rename the sources
    test = test.rename(index=rename_dict)
    test.to_csv("{}_replicates.csv".format(metric))
    display(test)

# for i in metrics:
#     print(i)
#     metric_df = replicates.pivot(index="Source", columns="Pipeline", values=i)
#     display(metric_df)

count                                    mean             \
Pipeline   biobakery3 biobakery4 jams wgsa2 woltka biobakery3 biobakery4   
Source                                                                     
Amos HiLo         5.0        5.0  5.0   5.0    5.0    0.00954   0.000000   
Amos Mixed        5.0        5.0  5.0   5.0    5.0    3.86412   0.000000   
Tourlousse        6.0        6.0  6.0   6.0    6.0   12.22505   4.716617   

                                                  std                       \
Pipeline        jams      wgsa2     woltka biobakery3 biobakery4      jams   
Source                                                                       
Amos HiLo   3.269400   4.065480  22.239680   0.021332   0.000000  0.115418   
Amos Mixed  9.755280  14.063520  27.391800   0.220681   0.000000  1.431971   
Tourlousse  9.789317  20.190933  26.144283   0.142976   0.037888  0.222829   

                                
Pipeline       wgsa2    woltka  
Source                          
Amos HiLo   0.096760  0.205998  
Amos Mixed  0.115006  0.307410  
Tourlousse  0.093907  0.248612

count                                    mean             \
Pipeline   biobakery3 biobakery4 jams wgsa2 woltka biobakery3 biobakery4   
Source                                                                     
Amos HiLo         5.0        5.0  5.0   5.0    5.0    89.4737   100.0000   
Amos Mixed        5.0        5.0  5.0   5.0    5.0    94.7368   100.0000   
Tourlousse        6.0        6.0  6.0   6.0    6.0    78.9474    94.7368   

                                               std                             \
Pipeline         jams    wgsa2   woltka biobakery3 biobakery4      jams wgsa2   
Source                                                                          
Amos HiLo    98.94736  89.4737  94.7368        0.0        0.0  2.353775   0.0   
Amos Mixed  100.00000  89.4737  94.7368        0.0        0.0  0.000000   0.0   
Tourlousse  100.00000  89.4737  94.7368        0.0        0.0  0.000000   0.0   

                   
Pipeline   woltka  
Source             
Amos HiLo     0.0  
Amos Mixed    0.0  
Tourlousse    0.0

count                                    mean             \
Pipeline   biobakery3 biobakery4 jams wgsa2 woltka biobakery3 biobakery4   
Source                                                                     
Amos HiLo         5.0        5.0  5.0   5.0    5.0   8.402520    2.71252   
Amos Mixed        5.0        5.0  5.0   5.0    5.0   8.767440    1.99916   
Tourlousse        6.0        6.0  6.0   6.0    6.0  14.567183    3.80855   

                                                  std                       \
Pipeline        jams      wgsa2     woltka biobakery3 biobakery4      jams   
Source                                                                       
Amos HiLo   12.79564  12.436960  16.384500   1.069797   0.040068  0.341404   
Amos Mixed  10.42296  14.815740  20.629920   0.774772   0.055893  0.358741   
Tourlousse   8.16020  14.378967  23.903417   0.725752   0.024836  0.396388   

                                
Pipeline       wgsa2    woltka  
Source                          
Amos HiLo   0.286740  0.112320  
Amos Mixed  0.051594  0.216708  
Tourlousse  0.100234  0.260645

In [13]:
one_to_one_copy = one_to_one.copy()
one_to_one_copy["Community"] = one_to_one["SampleID"] + " " + one_to_one["Source"]
# display(one_to_one_copy.head())
one_to_one_copy = one_to_one_copy[["FPRA", "Sens", "AD", "Community", "Pipeline"]]
# display(one_to_one_copy.head())

one_to_one_rename = {
    "EG nist": "NIST EG",
    "MIX-A nist": "NIST MIX-A",
    "MIX-B nist": "NIST MIX-B",
    "MIX-C nist": "NIST MIX-C",
    "MIX-D nist": "NIST MIX-D",
    "S1 bmock12": "BMock12",
    "S1 camisimGI": "CamiSim S1",
    "S2 camisimGI": "CamiSim S2",
}

# We want the five pipelines as the columns
one_to_one_copy = one_to_one_copy.pivot(index="Community", columns="Pipeline", values=["FPRA", "Sens", "AD"])
one_to_one_copy = one_to_one_copy.rename(index=one_to_one_rename)
display(one_to_one_copy)
one_to_one_copy.to_csv("one_to_one.csv", index=True)

FPRA                                             Sens  \
Pipeline   biobakery3 biobakery4     jams    wgsa2   woltka biobakery3   
Community                                                                
NIST EG        2.6645     0.0000   0.2104   1.1601  26.4802    85.7143   
NIST MIX-A     0.8464     0.0000   0.0520   0.6781  30.9152    81.8182   
NIST MIX-B    11.4154     0.0141   0.2655   1.0866  31.0568    72.7273   
NIST MIX-C    32.0567    32.3094  22.3335   9.1620  69.1682    63.6364   
NIST MIX-D     0.0607     0.1024   0.0194   0.2506  13.1948    72.7273   
BMock12        2.6303    17.9266  50.0438  37.3443  11.9647    36.3636   
CamiSim S1     0.5265     3.0566   1.4388   2.0432   8.5091   100.0000   
CamiSim S2     0.0586     3.8344   0.2055   3.9102  30.6668   100.0000   

                                                           AD             \
Pipeline   biobakery4      jams     wgsa2   woltka biobakery3 biobakery4   
Community                                                                  
NIST EG       92.8571   92.8571  100.0000  85.7143    10.1826     8.9446   
NIST MIX-A    90.9091   72.7273  100.0000  90.9091     9.5852     2.8426   
NIST MIX-B    72.7273   81.8182  100.0000  90.9091    13.6522     9.8140   
NIST MIX-C    63.6364   90.9091  100.0000  81.8182    14.1087    11.5690   
NIST MIX-D    72.7273   81.8182  100.0000  81.8182     4.1345     4.1098   
BMock12       63.6364   54.5455   45.4545   9.0909    16.6180    11.3065   
CamiSim S1    89.4737   97.3684   97.3684  86.8421     8.6188    12.7647   
CamiSim S2    95.2381  100.0000  100.0000  80.9524     1.2308     7.1138   

                                       
Pipeline       jams    wgsa2   woltka  
Community                              
NIST EG     12.2531  10.6638  22.5953  
NIST MIX-A  12.3527   6.9975  19.2420  
NIST MIX-B  13.5504  13.0792  22.8751  
NIST MIX-C  17.1601  14.7020  24.1026  
NIST MIX-D  11.2166   8.9316  21.1155  
BMock12     22.3681  24.4673  26.4484  
CamiSim S1  15.0361  13.4764  30.7596  
CamiSim S2   4.8525   6.3786  25.8766